CHAİN YAPISI

Simple Chain:

Kullanıcıdan bir bilgi istemi alacak ve bunu LLM'e iletecek ve size çıkış yanıtı verecektir.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.6,
)

In [3]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [6]:
llm=OpenAI()

In [7]:
prompt=PromptTemplate(
    input_variables=["place"],
    template="Best places to visit in {place?}"
)

In [8]:
chain=LLMChain(llm=llm,prompt=prompt)
print(chain.run("USA"))



1. New York City, New York
New York City is the most populous city in the United States and is known for its iconic landmarks such as the Statue of Liberty, Times Square, and the Empire State Building. It is also a cultural hub with world-renowned museums, Broadway shows, and diverse cuisine.

2. Grand Canyon National Park, Arizona
The Grand Canyon is a must-see for nature lovers, with its breathtaking views and hiking trails. Visitors can also take a helicopter tour or raft down the Colorado River to fully experience the beauty of this natural wonder.

3. San Francisco, California
San Francisco is a unique and vibrant city known for its iconic Golden Gate Bridge, cable cars, and diverse neighborhoods. It also offers beautiful views of the bay, delicious seafood, and a thriving arts and culture scene.

4. Las Vegas, Nevada
Las Vegas is a popular destination for its vibrant nightlife and world-renowned casinos. Visitors can also take in shows, fine dining, and shopping on the famous L

Simple Sequential Chains: 
Basit ardışık zincirler yani bir zincirin diğerine gerçek zamanlı olarak bağlanmamıza yardımcı olacaktır.

In [9]:
from langchain.chains import SimpleSequentialChain
from langchain import HuggingFaceHub

In [10]:
template=""" You have to suggest 5 best places to visit in {place}?
YOUR RESPONSE
"""
prompt_template=PromptTemplate(
    input_variables=["place"],
    template=template
)

In [11]:
HF_llm=HuggingFaceHub(repo_id="google/flan-t5-large")

/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
place_chain=LLMChain(llm=HF_llm,prompt=prompt_template)

In [14]:
template="""Given a list a places, please estimate the expenses to visit all of them in local curreny and also the days needed {expenses}
YOUR RESPONSE
"""
prompt_template=PromptTemplate(
    input_variables=["expenses"],
    template=template
)

In [15]:
llm=OpenAI()

In [16]:
expenses_chain=LLMChain(llm=llm,prompt=prompt_template)

In [17]:
Final_chain=SimpleSequentialChain(chains=[place_chain,expenses_chain],verbose=True)

In [18]:
review=Final_chain.run("USA")



> Entering new SimpleSequentialChain chain...
New York City

As an AI, I do not have access to current currency exchange rates. However, based on the average costs of visiting major tourist destinations in New York City, the estimated expenses for visiting all of the places could range from $1000 to $2000 per person. This includes accommodation, transportation, food, and attractions.

Additionally, the number of days needed to visit all the places in New York City would depend on the pace of the traveler and the number of places they wish to visit each day. However, a minimum of 5-7 days would be recommended to fully explore and experience the city.

> Finished chain.


load_summarize_chain

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.6,
)

In [21]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

In [22]:
llm= OpenAI()

In [23]:
with open("Sample.txt") as f:
    data=f.read()

In [24]:
text_splitter=CharacterTextSplitter()
texts=text_splitter.split_text(data)

In [25]:
docs=[Document(page_content=t)for t in texts]

In [26]:
docs

[Document(page_content="India, officially known as the Republic of India, is a diverse and vibrant country located in South Asia. With a rich history spanning thousands of years, India is known for its cultural heritage, religious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene backwaters of Kerala in the south, India encompasses a wide range of geographical features, including deserts, plains, mountains, and coastlines, making it a land of incredible natural beauty.\n\nIndia is the seventh-largest country by land area and the second-most populous country in the world, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic republic, with a president as the head of state and a prime minister as the head of government. The country follows a multi-tiered administrative structure, with 28 states and 9 union territories, each having its own elected government.\n\nIndia has a rich cultural heritage that has evolved over

In [27]:
chain=load_summarize_chain(llm,chain_type="map_reduce",verbose=True)

In [28]:
chain.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"India, officially known as the Republic of India, is a diverse and vibrant country located in South Asia. With a rich history spanning thousands of years, India is known for its cultural heritage, religious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene backwaters of Kerala in the south, India encompasses a wide range of geographical features, including deserts, plains, mountains, and coastlines, making it a land of incredible natural beauty.

India is the seventh-largest country by land area and the second-most populous country in the world, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic republic, with a president as the head of state and a prime minister as the head of government. The country follows a multi-tiered administrative structure, with 28 st

' India is a diverse and vibrant country with a rich history, cultural heritage, and a rapidly growing economy. It has a federal parliamentary democratic republic and is home to various religions that coexist harmoniously. Despite facing socio-economic challenges, India values education and has a thriving film industry and renowned cuisine. It is also a popular tourist destination and has made significant advancements in space exploration.'

HTTP Requests

LLMRequestsChain : Herhangi bir HTTP URLsine ulaşmamıza ve yanıtı geri döndürmeye yardımcı olacaktır.

In [35]:
from langchain.chains.llm_requests import LLMRequestsChain
from langchain.chains.llm import LLMChain

In [36]:
template="""Extract the answer to the question '{query}' or say "not found" if the information is not available.{requests_result}"""
PROMPT=PromptTemplate(
    input_variables=["query","requests_result"],
    template=template,
)

In [37]:
llm=OpenAI()

In [38]:
chain=LLMRequestsChain(llm_chain=LLMChain(llm=llm,prompt=PROMPT))

In [43]:
question="What is the capital of india?"
inputs={
    "query":question,
    "url":"https://www.google.com/search?q="+question.replace(" ","+"),
}

In [44]:
chain(inputs)

{'query': 'What is the capital of india?',
 'url': 'https://www.google.com/search?q=What+is+the+capital+of+india?',
 'output': " Ana içeriğe geç    Benzer  Sonuçlar   Yeni Delhi Wikipedia https://en.wikipedia.org › wiki › New_Delhi New Delhi is the capital of India and a part of the National Capital Territory of Delhi (NCT). New Delhi is the seat of all three branches of the Government of India,\xa0... New Delhi History, Population, Map, & Facts Britannica https://www.britannica.com › ... › Britannica › Places › Asia Britannica New Delhi, national capital of India. It is situated in the north-central part of the country on the west bank of the Yamuna River, adjacent to and just south of Delhi\xa0... New Delhi, India Image of the Week Earth Watching https://earth.esa.int › article European Space Agency New Delhi is a city in northern India, and capital of the country. It is located in the National Capital Territory of Delhi, on the Yamuna River. Delhi Wikipedia https://en.wikipedia.org 

In [45]:
import inspect
print (inspect.getsource(chain._call))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        from bs4 import BeautifulSoup

        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        # Other keys are assumed to be needed for LLM prediction
        other_keys = {k: v for k, v in inputs.items() if k != self.input_key}
        url = inputs[self.input_key]
        res = self.requests_wrapper.get(url)
        # extract the text from the html
        soup = BeautifulSoup(res, "html.parser")
        other_keys[self.requests_key] = soup.get_text()[: self.text_length]
        result = self.llm_chain.predict(  # type: ignore[attr-defined]
            callbacks=_run_manager.get_child(), **other_keys
        )
        return {self.output_key: result}

